In [17]:
import gretel
import pandas as pd
import numpy as np
from gretel_synthetics.timeseries_dgan.dgan import DGAN
from gretel_synthetics.timeseries_dgan.config import DGANConfig, OutputType, Normalization

# Load timeseries example to a dataframe


data_source = 'credit-timeseries-dataset.csv'
original_df = pd.read_csv(data_source)
original_df

,net_amt,date,account_balance,credit_amt,district_id,debit_amt
0,50425.0,1995-04-30,485849.7,232833.8,77,182408.8
1,22260.4,1995-03-31,435424.7,269788.6,77,247528.2
2,94953.0,1995-02-28,413164.3,272404.2,77,177451.2
3,13419.3,1994-11-30,476239.2,209552.5,77,196133.2
4,-109497.2,1995-01-31,318211.3,209838.0,77,319335.2
...,...,...,...,...,...,...
5539,320044.0,1996-02-29,997363.3,659139.6,4,339095.6
5540,80647.8,1995-12-31,836115.8,434639.6,4,353991.8
5541,-158796.5,1996-01-31,677319.3,408007.3,4,566803.8
5542,44954.8,1996-03-31,1042318.1,453294.0,4,408339.2


In [18]:
# Gretel Transforms Configuration
config = """
schema_version: "1.0"
models:
    - transforms:
        data_source: "__tmp__"
        policies:
            - name: shiftnumbers
              rules:
                - name: shiftnumbers
                  conditions:
                    field_name:
                        - account_balance
                        - credit_amt
                        - debit_amt
                        - net_amt
                  transforms:
                    - type: numbershift
                      attrs:
                        min: 1
                        max: 100
                        field_name:
                            - date
                            - district_id
"""

In [20]:
seed_fields = ["date", "district_id"]

task = {
    'type': 'seed',
    'attrs': {
        'fields': seed_fields,
    }
}

In [19]:
#Set up the seed fields
seed_fields = ["date", "district_id"]

task = {
    'type': 'seed',
    'attrs': {
        'fields': seed_fields,
    }
}

# Fine tune model parameters. These are the parameters we found to work best.  This is "Run 20" in the document
config['models'][0]['synthetics']['task'] = task

config['models'][0]['synthetics']['params']['vocab_size'] = 19
config['models'][0]['synthetics']['params']['learning_rate'] = 0.001
config['models'][0]['synthetics']['params']['epochs'] = 500
config['models'][0]['synthetics']['params']['dropout_rate'] = .5
config['models'][0]['synthetics']['params']['gen_temp'] = .8
config['models'][0]['synthetics']['generate']['num_records'] = train_df.shape[0]

TypeError: string indices must be integers